In [ ]:
import os 
import datetime
from pathlib import Path, PurePath, PosixPath
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
from skimage.transform import resize
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import resnet_v2
from tensorflow.keras.callbacks import TensorBoard
from src.utils.save_model import save_model, model_set
from src.datasets.generator import ImageDataGenerator

In [ ]:
# Load labels and split into training and valid

DATASET_FOLDER = Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY"))

data = pd.read_csv(DATASET_FOLDER / 'train.csv', index_col=[0])
data = data[data['Frontal/Lateral'].str.contains("Frontal")]
data = data[:256]
data_train, data_val = train_test_split(data, test_size=0.2)

In [ ]:
# create generators
columns = ['Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 'Edema']
train_datagen = ImageDataGenerator(data_train, DATASET_FOLDER, columns, batch_size=8, drop_last=True)
valid_datagen = ImageDataGenerator(data_train, DATASET_FOLDER, columns, batch_size=8)

In [ ]:
base_model = resnet_v2.ResNet152V2(include_top=False, weights='imagenet')

# add global pooling and dense output layer 
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
prediction_layer = Dense(len(columns), activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=prediction_layer)

In [ ]:

# compile model
adam = keras.optimizers.Adam()
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# fit model
num_epochs = 3
STEP_SIZE_TRAIN = len(data_train) // train_datagen.batch_size
STEP_SIZE_VALID = len(data_val) // valid_datagen.batch_size
result = model.fit_generator(generator=train_datagen,
                             steps_per_epoch=STEP_SIZE_TRAIN,
                             validation_data=valid_datagen,
                             validation_steps=STEP_SIZE_VALID,
                             epochs=num_epochs)